In [1]:
# Import dependencies
import pandas as pd
import requests
import time
import gmaps

# Import keys
from config import w_api_key
from config import gkey

In [2]:
# Load in and read City_Weather_data csv
weather_data = pd.read_csv("../WeatherPy/City_Weather_data.csv")
weather_data

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,saskylakh,71.9167,114.0833,-24.43,79,100,3.31,RU,1613249962
1,kapaa,22.0752,-159.3190,78.80,65,40,13.80,US,1613249755
2,diapaga,12.0708,1.7889,80.64,14,0,7.16,BF,1613249965
3,ushuaia,-54.8000,-68.3000,42.80,87,75,4.00,AR,1613249966
4,rundu,-17.9333,19.7667,64.00,77,89,4.00,NaN,1613249967
...,...,...,...,...,...,...,...,...,...
525,manjacaze,-24.7117,33.8828,75.81,93,100,7.18,MZ,1613250603
526,shelburne,44.0787,-80.2041,12.00,67,100,10.00,CA,1613250604
527,tulsipur,28.1310,82.2973,56.84,51,96,3.96,NP,1613250606
528,maihar,24.2667,80.7500,57.92,44,0,2.28,IN,1613250607


In [3]:
# Configure google heatmap
gmaps.configure(api_key=gkey)

# Store 'Lat' and 'Lng' into  locations 
locations = weather_data[["Lat", "Long"]].astype(float)

# Convert Poverty Rate to float and store
humidity = weather_data["Humidity"].astype(float)

In [4]:
# Create a humidity heatmap layer
fig = gmaps.figure(center=(0,0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=90,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# What are my preferred vacations conditions?
# 60-80 degrees
# Average cloudiness, 45-70??
# Humidty 50-65%

In [6]:
# Narrow down the dataframe to ideal conditions
ideal_weather = weather_data.loc[weather_data["Humidity"] >= 30]
ideal_weather = ideal_weather.loc[weather_data["Humidity"] <= 70]
ideal_weather = ideal_weather.loc[weather_data["Max Temp"] >= 55]
ideal_weather = ideal_weather.loc[weather_data["Max Temp"] <= 85]
ideal_weather = ideal_weather.loc[weather_data["Cloudiness"] >= 45]
ideal_weather = ideal_weather.loc[weather_data["Cloudiness"] <= 75]

ideal_weather["Nearby Hotel"] = ""

ideal_weather

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Nearby Hotel
63,los llanos de aridane,28.6585,-17.9182,66.20,63,75,12.66,ES,1613250050,
102,gameza,5.8026,-72.8059,59.79,68,73,4.70,CO,1613250101,
167,kavaratti,10.5669,72.6420,80.44,62,49,4.83,IN,1613250177,
256,peddapalli,18.6167,79.3667,65.93,52,62,5.32,IN,1613250282,
321,riachao das neves,-11.7461,-44.9100,80.69,67,69,2.30,BR,1613250360,
434,san roque,36.2107,-5.3842,60.80,55,72,2.30,ES,1613250492,
445,vryburg,-26.9566,24.7284,73.76,56,48,1.03,ZA,1613250505,
484,makakilo city,21.3469,-158.0858,82.40,57,75,8.05,US,1613250551,
498,sunyani,7.3399,-2.3268,83.61,59,45,7.94,GH,1613250567,


In [7]:
# Run test to ensure gkey is properly configured from sample city (from first city pull)

waycross = "31.2136, -82.3540"
search_type = "hotel"
radius = 5000


params = {
    "location": waycross,
    "radius": radius,
    "types": search_type,
    "key": gkey
}

# "location": waycross as test case

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
response = requests.get(base_url, params=params).json()
response["results"][1]["name"]
# Ok! Well, it is basically working.  So that's a good sign

'Super 8 by Wyndham Waycross GA'

In [8]:
# Find the first hotel within 5000 meters of each city

search_type = "hotel"
radius = 5000

params = {
    "radius": radius,
    "types": search_type,
    "key": gkey
}

for index, row in ideal_weather.iterrows():
    
    lat = row["Lat"]
    long = row["Long"]
    
    params["location"] = f"{lat},{long}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotel_returns = requests.get(base_url, params=params)
    
    hotel_returns = hotel_returns.json()
    
    print("Retrieving next hotel")
    
    try:
        ideal_weather.loc[index, "Nearby Hotel"] = hotel_returns["results"][1]["name"]
        
    except (KeyError, IndexError):
        print("No Hotel Available")
        ideal_weather.loc[index, "Nearby Hotel"] = np.nan
        pass
    
    time.sleep(1)
    

Retrieving next hotel
Retrieving next hotel
Retrieving next hotel
Retrieving next hotel
Retrieving next hotel
Retrieving next hotel
Retrieving next hotel
Retrieving next hotel
Retrieving next hotel


In [9]:
ideal_weather.dropna(inplace=True)
ideal_weather.reset_index(drop=True, inplace=True)
ideal_weather

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Nearby Hotel
0,los llanos de aridane,28.6585,-17.9182,66.20,63,75,12.66,ES,1613250050,Valle Aridane
1,gameza,5.8026,-72.8059,59.79,68,73,4.70,CO,1613250101,Estacion de Policia Gameza
2,kavaratti,10.5669,72.6420,80.44,62,49,4.83,IN,1613250177,All India Radio Kavaratti
3,peddapalli,18.6167,79.3667,65.93,52,62,5.32,IN,1613250282,State Bank of India - Peddapally Branch
4,riachao das neves,-11.7461,-44.9100,80.69,67,69,2.30,BR,1613250360,Prefeitura Municipal de Riachão das Neves
5,san roque,36.2107,-5.3842,60.80,55,72,2.30,ES,1613250492,Hotel Guadacorte Park
6,vryburg,-26.9566,24.7284,73.76,56,48,1.03,ZA,1613250505,Liquor Warehouse - Vryburg
7,makakilo city,21.3469,-158.0858,82.40,57,75,8.05,US,1613250551,Wet 'n' Wild Hawaii
8,sunyani,7.3399,-2.3268,83.61,59,45,7.94,GH,1613250567,Nass Lodge


In [10]:
locations_2 = ideal_weather[["Lat", "Long"]].astype(float)
humidity_2 = ideal_weather["Humidity"].astype(float)

info_box_template = """
<dl>
<dt>Name</dt><dd>{Nearby Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_weather.iterrows()]

fig_2 = gmaps.figure(center=(0,0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations_2, weights=humidity_2, 
                                 dissipating=False, max_intensity=70,
                                 point_radius = 1)

markers = gmaps.marker_layer(locations_2, info_box_content=hotel_info)

fig_2.add_layer(heat_layer)
fig_2.add_layer(markers)

fig_2

Figure(layout=FigureLayout(height='420px'))